In [1]:
#Fetch Data from website
!wget https://www.dropbox.com/s/n8bninud94blch4/fer2013.csv

--2020-06-03 19:05:52--  https://www.dropbox.com/s/n8bninud94blch4/fer2013.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/n8bninud94blch4/fer2013.csv [following]
--2020-06-03 19:05:52--  https://www.dropbox.com/s/raw/n8bninud94blch4/fer2013.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca5ea90b2d54d6aa0d10a9ab34e.dl.dropboxusercontent.com/cd/0/inline/A4_f-dIgmI18KpnJWo_l2eA3j54WrAoF_X-EigTtaDHdQ2WSnpmKHkLE0evQiEIWcELXssTgBqKIPvft5DFK9WZSNUfuhbgKTY3IZ-ZLJjDP_wtZBfnYosuklMFXavO3s44/file# [following]
--2020-06-03 19:05:52--  https://uca5ea90b2d54d6aa0d10a9ab34e.dl.dropboxusercontent.com/cd/0/inline/A4_f-dIgmI18KpnJWo_l2eA3j54WrAoF_X-EigTtaDHdQ2WSnpmKHkLE0evQiEIWcELXssTgBqKIPvft5DFK9WZSNUfuhbgKTY3IZ-ZLJjDP_wtZBfnYo

## Loading Pandas Dataframe

In [2]:
%tensorflow_version 1.x
import pandas as pd
import numpy as np
df = pd.read_csv('fer2013.csv')
df.describe()

TensorFlow 1.x selected.


,emotion
count,35887.000000
mean,3.323265
std,1.873819
min,0.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,6.000000


In [3]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
df['Usage'].unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

## Formating data
Q1: Reshape and Normalize the data

Q2: Normalize & Train_Test Split


In [5]:
from sklearn.model_selection import train_test_split
# Taking entire set irrespective of classes and using train_test_split 
X = np.array(list(map(str.split, df.pixels)), np.float32)
X = X.astype('float')
X = X/255

y=np.array(df['emotion'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10, random_state=42)

#print(len(X_train), len(y_test))
print("X_train.shape = ", X_train.shape)
print("y_test.shape = ", y_test.shape)

from tensorflow.keras.utils import to_categorical
num_classes = 7 # clases as per dataset (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad,5=Surprise, 6=Neutral)
#y_nc_train,y_nc_test = y_train, y_test # backup non-categorical data :)
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test, num_classes)

img_rows,img_cols = 48,48
X_train = X_train.reshape(X_train.shape[0],img_rows, img_cols,1)
X_test = X_test.reshape(X_test.shape[0],img_rows, img_cols,1)
input_shape = (img_rows, img_cols, 1)


X_train.shape =  (32298, 2304)
y_test.shape =  (3589,)


In [6]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras
from keras.models import Sequential
#from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from keras.utils import normalize, to_categorical
from keras.layers import Activation, Dropout

Using TensorFlow backend.


## Creating CNN keras model
Q3: Define CNN Model

In [7]:
model = Sequential() #keras model

#Layer 1
model.add(Conv2D(64,kernel_size = (5,5),padding="same", input_shape=(input_shape)))
model.add(Conv2D(64,kernel_size = (3,3),padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Activation('relu'))
#model.summary()

#Layer 2
model.add(Conv2D(128,kernel_size = (5,5),padding="same"))
model.add(Conv2D(128,kernel_size = (5,5),padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Activation('relu'))
#model.summary()

#Layer 3
model.add(Conv2D(256,kernel_size =(3,3),padding="same"))
model.add(Conv2D(256,kernel_size =(3,3),padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Activation('relu'))
#model.summary()

#Layer 4
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes,activation='softmax'))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



## Plotting model and list model summary

In [8]:
keras.utils.plot_model(model, "my_first_model.png",show_shapes=True)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1664      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 128)      

## Setting up model to train.
Q4: Fit CNN Model

In [9]:
model.compile(loss='categorical_crossentropy', # Cross-entropy
          optimizer=keras.optimizers.Adam(learning_rate=0.001), # Adam optimizer lr=1e-3
          metrics=['accuracy']) # Accuracy performance metric

model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split=0.2)


Train on 25838 samples, validate on 6460 samples
Epoch 1/20
25838/25838 [==============================] - 39s 1ms/step - loss: 1.6518 - accuracy: 0.3646 - val_loss: 2.2993 - val_accuracy: 0.3356
Epoch 2/20
25838/25838 [==============================] - 34s 1ms/step - loss: 1.3237 - accuracy: 0.4935 - val_loss: 1.3202 - val_accuracy: 0.4972
Epoch 3/20
25838/25838 [==============================] - 34s 1ms/step - loss: 1.2024 - accuracy: 0.5452 - val_loss: 1.3968 - val_accuracy: 0.4870
Epoch 4/20
25838/25838 [==============================] - 34s 1ms/step - loss: 1.1386 - accuracy: 0.5723 - val_loss: 1.3902 - val_accuracy: 0.4706
Epoch 5/20
25838/25838 [==============================] - 34s 1ms/step - loss: 1.0858 - accuracy: 0.5896 - val_loss: 1.2032 - val_accuracy: 0.5478
Epoch 6/20
25838/25838 [==============================] - 34s 1ms/step - loss: 1.0426 - accuracy: 0.6071 - val_loss: 1.3246 - val_accuracy: 0.5183
Epoch 7/20
25838/25838 [==============================] - 34s 1ms/st

## Confusion matrix
Q5: Draw confusion matrix for the trained model.

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

#Confution Matrix and Classification Report
Y_pred = model.predict_classes(X_test)
rounded_y=np.argmax(y_test, axis=1)

print('Confusion Matrix')
print(confusion_matrix(rounded_y, Y_pred))

print('Classification Report')
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad','Surprise', 'Neutral']
print(classification_report(rounded_y, Y_pred, target_names=labels))

Confusion Matrix
[[186   1 108  35  88  17  63]
 [ 10  19   7   1  11   1   3]
 [ 36   1 251  31 124  55  47]
 [ 16   0  40 716  44  27  38]
 [ 42   1  97  42 308   9  89]
 [  3   0  56  25  15 304  11]
 [ 28   0  81  59 139  14 290]]
Classification Report
              precision    recall  f1-score   support

       Angry       0.58      0.37      0.45       498
     Disgust       0.86      0.37      0.51        52
        Fear       0.39      0.46      0.42       545
       Happy       0.79      0.81      0.80       881
         Sad       0.42      0.52      0.47       588
    Surprise       0.71      0.73      0.72       414
     Neutral       0.54      0.47      0.50       611

    accuracy                           0.58      3589
   macro avg       0.61      0.53      0.56      3589
weighted avg       0.59      0.58      0.58      3589



## Saving/Loading the model Keras & tensorflow
Q6: Save the model to external .h5 File

Q7: Convert the saved model to .pb model

In [11]:
from keras import backend as K

# Save the model of faces & emotions in h5 format
model.save('model-nn/faces_emotions_model.h5')

# This line must be executed before loading Keras model.
K.set_learning_phase(0)

# Loading the saved model to new instance.
model_new = keras.models.load_model('model-nn/faces_emotions_model.h5')
print(model_new.inputs)
print(model_new.outputs)

[<tf.Tensor 'conv2d_1_input_1:0' shape=(?, 48, 48, 1) dtype=float32>]
[<tf.Tensor 'dense_2_1/Softmax:0' shape=(?, 7) dtype=float32>]


In [12]:
## reference : https://www.dlology.com/blog/how-to-convert-trained-keras-model-to-tensorflow-and-make-prediction/
import tensorflow as tf
from tensorflow.python.framework.graph_util import convert_variables_to_constants

output_names = [out.op.name for out in model.outputs]
keep_var_names=None
clear_devices=True
sess = K.get_session()
graph = sess.graph
frozen_graph = []
with graph.as_default():
  freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
  print(freeze_var_names)
  output_names = output_names or []
  output_names += [v.op.name for v in tf.global_variables()]
  # Graph -> GraphDef ProtoBuf
  input_graph_def = graph.as_graph_def()
  if clear_devices:
      for node in input_graph_def.node:
          node.device = ""
  frozen_graph = convert_variables_to_constants(sess, input_graph_def, output_names, freeze_var_names)

# Save to .pb format
tf.train.write_graph(frozen_graph, "model-nn", "faces_emotions_model.pb", as_text=False)


['conv2d_2/bias', 'batch_normalization_3_1/moving_mean', 'training/Adam/m_20_1', 'training/Adam/m_13_1', 'training/Adam/vhat_15_1', 'conv2d_3/bias', 'dense_1_1/bias', 'training_1/Adam/v_7_1', 'batch_normalization_2_1/gamma', 'training/Adam/vhat_0_1', 'training_1/Adam/vhat_21_1', 'conv2d_4_1/kernel', 'batch_normalization_3_1/moving_variance', 'training/Adam/vhat_3_1', 'batch_normalization_4/beta', 'training_1/Adam/m_8_1', 'conv2d_6/bias', 'training/Adam/m_9_1', 'training/Adam/m_18_1', 'training/Adam/m_10_1', 'conv2d_6_1/bias', 'training_1/Adam/m_17_1', 'training_1/Adam/m_1_1', 'training/Adam/v_15_1', 'training/Adam/vhat_14_1', 'dense_2_1/bias', 'training/Adam/vhat_12_1', 'training_1/Adam/vhat_3_1', 'training/Adam/vhat_22_1', 'training_1/Adam/m_3_1', 'training/Adam/m_7_1', 'training/Adam/m_8_1', 'training/Adam/vhat_21_1', 'training_1/Adam/v_5_1', 'training_1/Adam/vhat_16_1', 'training_1/Adam/vhat_7_1', 'training_1/Adam/vhat_18_1', 'training/Adam/m_6_1', 'batch_normalization_1/moving_mean

'model-nn/faces_emotions_model.pb'